In [2]:
import sys
import os
import os.path as op
sys.path.append(os.path.abspath('../catconv'))
import catconv.stabi as sb
import catconv.operations as co

In [3]:
import ujson
with open('data/SB_indicies.json', 'rb') as index_file:
    indexed = ujson.load(index_file)

In [15]:
sorted(indexed.keys())

[u'SB0010001',
 u'SB0010051',
 u'SB0010100',
 u'SB0010132',
 u'SB0010159',
 u'SB0010209',
 u'SB0010259',
 u'SB0010291',
 u'SB0010341',
 u'SB0010391',
 u'SB0010426',
 u'SB0010476',
 u'SB0010504',
 u'SB0010554',
 u'SB0010598',
 u'SB0010648',
 u'SB0010698',
 u'SB0010748',
 u'SB0010788',
 u'SB0010838',
 u'SB0010888',
 u'SB0010938',
 u'SB0010988',
 u'SB0020001',
 u'SB0020051',
 u'SB0020101',
 u'SB0020151',
 u'SB0020201',
 u'SB0020225',
 u'SB0020275',
 u'SB0020277',
 u'SB0020298',
 u'SB0020348',
 u'SB0020398',
 u'SB0020417',
 u'SB0020467',
 u'SB0020520',
 u'SB0020570',
 u'SB0020587',
 u'SB0020637',
 u'SB0020687',
 u'SB0020709',
 u'SB0020759',
 u'SB0020773',
 u'SB0020823',
 u'SB0030024',
 u'SB0030074',
 u'SB0030108',
 u'SB0030158',
 u'SB0030197',
 u'SB0030247',
 u'SB0030297',
 u'SB0030311',
 u'SB0030361',
 u'SB0030411',
 u'SB0030465',
 u'SB0030515',
 u'SB0030542',
 u'SB0030592',
 u'SB0030642',
 u'SB0030695',
 u'SB0030745',
 u'SB0040033',
 u'SB0040083',
 u'SB0040095',
 u'SB0040145',
 u'SB00401

In [10]:
old = sb.load_catalog('/media/jakob/bigdata/old_cat/SB',selection={'ext':'.tif'})

In [11]:
len(old['pages'])

191385

In [22]:
test_page = old['pages'][0]

In [23]:
test_page

{'path': '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000001.tif'}

In [26]:
indexed_pages = [page for page in old['pages'] if simple_id(sb.convert_page_path(page,{})['path']) in indexed]

In [18]:
def simple_id(path):
    data_dir, cat_name, batch_name, page_name = sb.split_path(path)
    return batch_name + page_name[4:]


In [48]:
old_to_index = [(page, sb.convert_page_path(page,{'to_cat':'/media/jakob/bigdata/train/','ext':'.png'})) for page in indexed_pages]

In [49]:
sb.change_path

<function catconv.stabi.change_path>

In [56]:
jobs = [co.crop(fpage['path'], tpage['path']) for fpage, tpage in old_to_index]

In [59]:
for pair in old_to_index:
    co.convert_to_png(*pair)

In [57]:
ex

[['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000001.tif',
  '-crop',
  '-0-70',
  '/media/jakob/bigdata/train/SB/SB001/00000001.png'],
 ['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000051.tif',
  '-crop',
  '-0-70',
  '/media/jakob/bigdata/train/SB/SB001/00000051.png'],
 ['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000100.tif',
  '-crop',
  '-0-70',
  '/media/jakob/bigdata/train/SB/SB001/00000100.png'],
 ['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000132.tif',
  '-crop',
  '-0-70',
  '/media/jakob/bigdata/train/SB/SB001/00000132.png'],
 ['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000159.tif',
  '-crop',
  '-0-70',
  '/media/jakob/bigdata/train/SB/SB001/00000159.png'],
 ['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000209.tif',
  '-crop',
  '-0-70',
  '/media/jakob/bigdata/train/SB/SB001/00000209.png'],
 ['convert',
  '/media/jakob/bigdata/old_cat/SB/SB001/TIF/00000259.tif',
  '-crop',
  '-0-70',
  '/m

In [61]:
indexed['SB0340335']

u'Bote Bock Jahre hundertfuenfundzwanzig'

In [5]:
import tinydb
from tinydb import Query
db = tinydb.TinyDB('../ocr-web-prototype/index_manual.json')

In [6]:
Page = Query()

In [7]:

line_texts = [(op.join(page['name'], (page['index_name'][0])), page['text'][0]) 
 for page in db.search(Page.name != None)
 if 'index_name' in page]

In [8]:
import codecs
import shutil
export = '/media/jakob/bigdata/train/SB_indexed'
catalog = '/media/jakob/bigdata/train/SB'
print len(line_texts)
for line, text in line_texts:
    gt_file  = op.join(export,  line + '.gt.txt')
    line_src = op.join(catalog, line + '.bin.png' )
    line_to  = op.join(export,  line + '.dew.png' )
    if not op.isdir(op.dirname(line_to)):
        os.makedirs(op.dirname(line_to))
    shutil.copyfile(line_src,line_to)
#     print gt_file
    with codecs.open(gt_file, 'wb', encoding='utf8') as gt:
        gt.write(text)

533


In [53]:
!cat /media/jakob/bigdata/train/SB/SB001/00000554/010001.gt.txt
# !ls -la /media/jakob/bigdata/train/SB/SB001/00000001/
!eog /media/jakob/bigdata/train/SB/SB001/00000554/010001.bin.png

Abrányi, Kornél

In [30]:
db.search(Page.name != None)

[{u'index_name': [u'010001'], u'name': u'SB001/00000001', u'text': [u'A A']},
 {u'index_name': [u'010001'],
  u'name': u'SB074/00000637',
  u'text': [u'Frotscher, Gotthold']},
 {u'index_name': [u'010001'],
  u'name': u'SB075/00000003',
  u'text': [u'Fuchs, Albert']},
 {u'index_name': [u'010001'],
  u'name': u'SB075/00000161',
  u'text': [u'Fuehrer, Rudolf H.']},
 {u'index_name': [u'010001'],
  u'name': u'SB001/00000476',
  u'text': [u'Abraham, Gerald [Mitarb.]']},
 {u'index_name': [u'010001'],
  u'name': u'SB236/00000319',
  u'text': [u'Zeno, Apostolo']},
 {u'index_name': [u'010001'],
  u'name': u'SB236/00000346',
  u'text': [u'Zepler, Bogumil']},
 {u'index_name': [u'010001'],
  u'name': u'SB236/00000269',
  u'text': [u'Sinfonietta']},
 {u'index_name': [u'010001'],
  u'name': u'SB236/00000396',
  u'text': [u'Zesso, Giovanni Battista']},
 {u'index_name': [u'010003'],
  u'name': u'SB236/00000446',
  u'text': [u'Ziegler, Hans. - 50 Jahre Basler Orchester-Gesellschaft.']},
 {u'index_name':